# Memory Tools: Giving the LLM Control Over Memory

## Introduction

In this advanced notebook, you'll learn how to give your agent control over its own memory using tools. Instead of automatically extracting memories, you can let the LLM decide what to remember and when to search for memories. The Agent Memory Server SDK provides built-in memory tools for this.

### What You'll Learn

- Why give the LLM control over memory
- Agent Memory Server's built-in memory tools
- How to configure memory tools for your agent
- When the LLM decides to store vs. search memories
- Best practices for memory-aware agents

### Prerequisites

- Completed all Section 3 notebooks
- Redis 8 running locally
- Agent Memory Server running
- OpenAI API key set

## Concepts: Tool-Based Memory Management

### Two Approaches to Memory

#### 1. Automatic Memory (What We've Been Doing)

```python
# Agent has conversation
# → Save working memory
# → Agent Memory Server automatically extracts important facts
# → Facts stored in long-term memory
```

**Pros:**
- ✅ Fully automatic
- ✅ No LLM overhead in your application
- ✅ Consistent extraction

**Cons:**
- ⚠️ Your application's LLM can't directly control what gets extracted
- ⚠️ May extract too much or too little
- ⚠️ Can't dynamically decide what's important based on conversation context

**Note:** You can configure custom extraction prompts on the memory server to guide what gets extracted, but your client application's LLM doesn't have direct control over the extraction process.

#### 2. Tool-Based Memory (This Notebook)

```python
# Agent has conversation
# → LLM decides: "This is important, I should remember it"
# → LLM calls store_memory tool
# → Fact stored in long-term memory

# Later...
# → LLM decides: "I need to know about the user's preferences"
# → LLM calls search_memories tool
# → Retrieves relevant memories
```

**Pros:**
- ✅ Your application's LLM has full control
- ✅ Can decide what's important in real-time
- ✅ Can search when needed
- ✅ More intelligent, context-aware behavior

**Cons:**
- ⚠️ Requires tool calls (more tokens)
- ⚠️ LLM might forget to store/search
- ⚠️ Less consistent

### When to Use Tool-Based Memory

**Use tool-based memory when:**
- ✅ Agent needs fine-grained control
- ✅ Importance is context-dependent
- ✅ Agent should decide when to search
- ✅ Building advanced, autonomous agents

**Use automatic memory when:**
- ✅ Simple, consistent extraction is fine
- ✅ Want to minimize token usage
- ✅ Building straightforward agents

**Best: Use both!**
- Automatic extraction for baseline
- Tools for explicit control

### Agent Memory Server's Built-in Tools

The Agent Memory Server SDK provides:

1. **`store_memory`** - Store important information
2. **`search_memories`** - Search for relevant memories
3. **`update_memory`** - Update existing memories
4. **`delete_memory`** - Remove memories

These are pre-built, tested, and optimized!

## Setup

In [ ]:
import os
import asyncio
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from pydantic import BaseModel, Field
from typing import List, Optional
from agent_memory_client import MemoryAPIClient as MemoryClient, MemoryClientConfig

# Initialize
student_id = "student_memory_tools"
session_id = "tool_demo"

# Initialize memory client with proper config
import os
config = MemoryClientConfig(
    base_url=os.getenv("AGENT_MEMORY_URL", "http://localhost:8000"),
    default_namespace="redis_university"
)
memory_client = MemoryClient(config=config)

llm = ChatOpenAI(model="gpt-4o", temperature=0.7)

print(f"✅ Setup complete for {student_id}")

## Exploring Agent Memory Server's Memory Tools

Let's create tools that wrap the Agent Memory Server's memory operations.

### Tool 1: Store Memory

In [ ]:
class StoreMemoryInput(BaseModel):
    text: str = Field(description="The information to remember")
    memory_type: str = Field(
        default="semantic",
        description="Type of memory: 'semantic' for facts, 'episodic' for events"
    )
    topics: List[str] = Field(
        default=[],
        description="Topics/tags for this memory (e.g., ['preferences', 'courses'])"
    )

@tool(args_schema=StoreMemoryInput)
async def store_memory(text: str, memory_type: str = "semantic", topics: List[str] = []) -> str:
    """
    Store important information in long-term memory.
    
    Use this tool when:
    - Student shares preferences (e.g., "I prefer online courses")
    - Student states goals (e.g., "I want to graduate in 2026")
    - Student provides important facts (e.g., "My major is Computer Science")
    - You learn something that should be remembered for future sessions
    
    Do NOT use for:
    - Temporary conversation context (working memory handles this)
    - Trivial details
    - Information that changes frequently
    
    Examples:
    - text="Student prefers morning classes", memory_type="semantic", topics=["preferences", "schedule"]
    - text="Student completed CS101 with grade A", memory_type="episodic", topics=["courses", "grades"]
    """
    try:
        await memory_client.create_long_term_memory([ClientMemoryRecord(
            text=text,
            memory_type=memory_type,
            topics=topics if topics else ["general"]
        )])
        return f"✅ Stored memory: {text}"
    except Exception as e:
        return f"❌ Failed to store memory: {str(e)}"

print("✅ store_memory tool defined")

### Tool 2: Search Memories

In [ ]:
class SearchMemoriesInput(BaseModel):
    query: str = Field(description="What to search for in memories")
    limit: int = Field(default=5, description="Maximum number of memories to retrieve")

@tool(args_schema=SearchMemoriesInput)
async def search_memories(query: str, limit: int = 5) -> str:
    """
    Search for relevant memories using semantic search.
    
    Use this tool when:
    - You need to recall information about the student
    - Student asks "What do you know about me?"
    - You need context from previous sessions
    - Making personalized recommendations
    
    The search uses semantic matching, so natural language queries work well.
    
    Examples:
    - query="student preferences" → finds preference-related memories
    - query="completed courses" → finds course completion records
    - query="goals" → finds student's stated goals
    """
    try:
        memories = await memory_client.search_long_term_memory(
            text=query,
            limit=limit
        )
        
        if not memories:
            return "No relevant memories found."
        
        result = f"Found {len(memories)} relevant memories:\n\n"
        for i, memory in enumerate(memories, 1):
            result += f"{i}. {memory.text}\n"
            result += f"   Type: {memory.memory_type} | Topics: {', '.join(memory.topics)}\n\n"
        
        return result
    except Exception as e:
        return f"❌ Failed to search memories: {str(e)}"

print("✅ search_memories tool defined")

## Testing Memory Tools with an Agent

Let's create an agent that uses these memory tools.

In [ ]:
# Configure agent with memory tools
memory_tools = [store_memory, search_memories]
llm_with_tools = llm.bind_tools(memory_tools)

system_prompt = """You are a class scheduling agent for Redis University.

You have access to memory tools:
- store_memory: Store important information about the student
- search_memories: Search for information you've stored before

Use these tools intelligently:
- When students share preferences, goals, or important facts → store them
- When you need to recall information → search for it
- When making recommendations → search for preferences first

Be proactive about using memory to provide personalized service.
"""

print("✅ Agent configured with memory tools")

### Example 1: Agent Stores a Preference

In [ ]:
print("=" * 80)
print("EXAMPLE 1: Agent Stores a Preference")
print("=" * 80)

user_message = "I prefer online courses because I work part-time."

messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_message)
]

print(f"\n👤 User: {user_message}")

# First response - should call store_memory
response = llm_with_tools.invoke(messages)

if response.tool_calls:
    print("\n🤖 Agent decision: Store this preference")
    for tool_call in response.tool_calls:
        print(f"   Tool: {tool_call['name']}")
        print(f"   Args: {tool_call['args']}")
        
        # Execute the tool
        if tool_call['name'] == 'store_memory':
            result = await store_memory.ainvoke(tool_call['args'])
            print(f"   Result: {result}")
            
            # Add tool result to messages
            messages.append(response)
            messages.append(ToolMessage(
                content=result,
                tool_call_id=tool_call['id']
            ))
    
    # Get final response
    final_response = llm_with_tools.invoke(messages)
    print(f"\n🤖 Agent: {final_response.content}")
else:
    print(f"\n🤖 Agent: {response.content}")
    print("\n⚠️  Agent didn't use store_memory tool")

print("\n" + "=" * 80)

### Example 2: Agent Searches for Memories

In [ ]:
print("\n" + "=" * 80)
print("EXAMPLE 2: Agent Searches for Memories")
print("=" * 80)

# Wait a moment for memory to be stored
await asyncio.sleep(1)

user_message = "What courses would you recommend for me?"

messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_message)
]

print(f"\n👤 User: {user_message}")

# First response - should call search_memories
response = llm_with_tools.invoke(messages)

if response.tool_calls:
    print("\n🤖 Agent decision: Search for preferences first")
    for tool_call in response.tool_calls:
        print(f"   Tool: {tool_call['name']}")
        print(f"   Args: {tool_call['args']}")
        
        # Execute the tool
        if tool_call['name'] == 'search_memories':
            result = await search_memories.ainvoke(tool_call['args'])
            print(f"\n   Retrieved memories:")
            print(f"   {result}")
            
            # Add tool result to messages
            messages.append(response)
            messages.append(ToolMessage(
                content=result,
                tool_call_id=tool_call['id']
            ))
    
    # Get final response
    final_response = llm_with_tools.invoke(messages)
    print(f"\n🤖 Agent: {final_response.content}")
    print("\n✅ Agent used memories to personalize recommendation!")
else:
    print(f"\n🤖 Agent: {response.content}")
    print("\n⚠️  Agent didn't search memories")

print("\n" + "=" * 80)

### Example 3: Multi-Turn Conversation with Memory

In [ ]:
print("\n" + "=" * 80)
print("EXAMPLE 3: Multi-Turn Conversation")
print("=" * 80)

async def chat_with_memory(user_message, conversation_history):
    """Helper function for conversation with memory tools."""
    messages = [SystemMessage(content=system_prompt)]
    messages.extend(conversation_history)
    messages.append(HumanMessage(content=user_message))
    
    # Get response
    response = llm_with_tools.invoke(messages)
    
    # Handle tool calls
    if response.tool_calls:
        messages.append(response)
        
        for tool_call in response.tool_calls:
            # Execute tool
            if tool_call['name'] == 'store_memory':
                result = await store_memory.ainvoke(tool_call['args'])
            elif tool_call['name'] == 'search_memories':
                result = await search_memories.ainvoke(tool_call['args'])
            else:
                result = "Unknown tool"
            
            messages.append(ToolMessage(
                content=result,
                tool_call_id=tool_call['id']
            ))
        
        # Get final response after tool execution
        response = llm_with_tools.invoke(messages)
    
    # Update conversation history
    conversation_history.append(HumanMessage(content=user_message))
    conversation_history.append(AIMessage(content=response.content))
    
    return response.content, conversation_history

# Have a conversation
conversation = []

queries = [
    "I'm a junior majoring in Computer Science.",
    "I want to focus on machine learning and AI.",
    "What do you know about me so far?",
]

for query in queries:
    print(f"\n👤 User: {query}")
    response, conversation = await chat_with_memory(query, conversation)
    print(f"🤖 Agent: {response}")
    await asyncio.sleep(1)

print("\n" + "=" * 80)
print("✅ Agent proactively stored and retrieved memories!")

## Key Takeaways

### Benefits of Memory Tools

✅ **LLM Control:**
- Agent decides what's important
- Agent decides when to search
- More intelligent behavior

✅ **Flexibility:**
- Can store context-dependent information
- Can search on-demand
- Can update/delete memories

✅ **Transparency:**
- You can see when agent stores/searches
- Easier to debug
- More explainable

### When to Use Memory Tools

**Use memory tools when:**
- ✅ Building advanced, autonomous agents
- ✅ Agent needs fine-grained control
- ✅ Importance is context-dependent
- ✅ Want explicit memory operations

**Use automatic extraction when:**
- ✅ Simple, consistent extraction is fine
- ✅ Want to minimize token usage
- ✅ Building straightforward agents

**Best practice: Combine both!**
- Automatic extraction as baseline
- Tools for explicit control

### Tool Design Best Practices

1. **Clear descriptions** - Explain when to use each tool
2. **Good examples** - Show typical usage
3. **Error handling** - Handle failures gracefully
4. **Feedback** - Return clear success/failure messages

### Common Patterns

**Store after learning:**
```
User: "I prefer online courses"
Agent: [stores memory] "Got it, I'll remember that!"
```

**Search before recommending:**
```
User: "What courses should I take?"
Agent: [searches memories] "Based on your preferences..."
```

**Proactive recall:**
```
User: "Tell me about CS401"
Agent: [searches memories] "I remember you're interested in ML..."
```

## Exercises

1. **Test memory decisions**: Have a 10-turn conversation. Does the agent store and search appropriately?

2. **Add update tool**: Create an `update_memory` tool that lets the agent modify existing memories.

3. **Compare approaches**: Build two agents - one with automatic extraction, one with tools. Which performs better?

4. **Memory strategy**: Design a system prompt that guides the agent on when to use memory tools.

## Summary

In this notebook, you learned:

- ✅ Memory tools give the LLM control over memory operations
- ✅ Agent Memory Server provides built-in memory tools
- ✅ Tools enable intelligent, context-aware memory management
- ✅ Combine automatic extraction with tools for best results
- ✅ Clear tool descriptions guide proper usage

**Key insight:** Tool-based memory management enables more sophisticated agents that can decide what to remember and when to recall information. This is especially powerful for autonomous agents that need fine-grained control over their memory.